In [1]:
%load_ext autoreload
%autoreload 2

In [15]:
import sys
import os
import torch
from functools import partial
from torch.utils.data import Dataset, DataLoader

import torch.nn as nn
from embedding import CustomEmbedding
from transformers import EncoderDecoderTransformer
from utils import padding_collate_fn

from generate_data import RandomIntegerDataset

In [16]:
N_WORDS = 10
PAD_TOKEN_IDX = N_WORDS
SOS_TOKEN_IDX = N_WORDS + 1
EOS_TOKEN_IDX = N_WORDS + 2
vocab_size = N_WORDS + 3
D_MODEL = 16

embeddings = CustomEmbedding(vocab_size = vocab_size, d_model = D_MODEL) # 3 = PAD, SOS, EOS

indices = torch.tensor([1,9])

# print(embeddings.embeddings.weight)
print(embeddings.embeddings(indices))

tensor([[-1.3423,  0.7971, -0.9860, -0.9616, -0.0817,  0.8046, -0.8727,  0.9604,
          0.9754,  0.2429,  1.7459, -0.2746, -0.2651, -0.1627, -1.3972,  0.2668],
        [-1.0464, -0.2298, -0.7569,  0.7564, -1.9449,  2.0051, -0.0415,  0.5735,
         -1.5901, -0.8665, -0.5932, -1.5176,  0.7348,  0.9515, -0.2090,  2.0376]],
       grad_fn=<EmbeddingBackward0>)


In [32]:
BATCH_SIZE = 2

rand_ds = RandomIntegerDataset(2, 10, 10, [i for i in range(10)])
dataloader = DataLoader(rand_ds, batch_size = BATCH_SIZE, shuffle = True, collate_fn = partial(padding_collate_fn, pad_token_idx = PAD_TOKEN_IDX))

In [35]:
input, label = next(iter(dataloader))
print(input[0])
print(input[1])
print(label)

tensor([[ 5,  8,  3,  2,  8,  8,  1],
        [ 2,  5, 10, 10, 10, 10, 10]])
tensor([[11,  1,  2,  3,  5,  8,  8,  8],
        [11,  2,  5, 10, 10, 10, 10, 10]])
tensor([[ 1,  2,  3,  5,  8,  8,  8, 12],
        [ 2,  5, 12, 10, 10, 10, 10, 10]])


In [37]:
loss_fn = nn.CrossEntropyLoss()

model = EncoderDecoderTransformer(
                    embeddings = embeddings, 
                    vocab_size = vocab_size, 
                    d_model = D_MODEL, 
                    num_attention_heads = 4, 
                    num_encoder_layers = 6, 
                    num_decoder_layers = 6, 
                    dim_feedforward = 64, 
                    max_context_window = 50)

optim = torch.optim.SGD(params = model.parameters(), lr = 1e-3, momentum = 0.9, weight_decay = 1e-4)

In [29]:
# def run_train_epoch(dataloader: DataLoader, model: nn.Module, loss_fn: nn.Module, optimizer: torch.optim.Optimizer):
    
#     model.train()

#     for i, ((source, target), label) in enumerate(dataloader):

#         print(source)
#         print(target)

#         # FORWARD
#         pred = model(source, target)

#         batch_loss = loss_fn(pred, label)

#         # BACKWARD
#         batch_loss.backward()

#         # OPTIMIZE
#         optimizer.step()
#         optimizer.zero_grad()

# def run_test_loop(dataloader: DataLoader, model: nn.Module, loss_fn: nn.Module):
    
#     model.eval()

In [30]:
# EPOCHS = 1

# for i in range(EPOCHS):

#     run_train_epoch(dataloader, model, loss_fn, optim)